# Part IIA: Explicitly solving Tic-Tac-Toe with the Minimax Algorithm

- To know more about the Minimax algorithm, you can read [the notes I have prepared](../report.pdf) or the [Wikipedia article on it](https://en.wikipedia.org/wiki/Minimax). It is a very general tool for solving decision-making problems, however, it can become very computationally expensive. We will analyze this algorithm for the special case of finite terminating two player games.

- Tic-Tac-Toe is a finite terminating two player game that can only end in a loss for one player (and a win for the other) or in a tie. It can be shown that for every such game, at least one of the players has a non-losing strategy (some of you may see this proof again later in CS 207 - it's basically structural induction over Game Trees). This includes games such as Chess!

- The algorithm is based on forming the game tree: The root of this tree is the starting position, and the children of each node are the positions reachable from the node position. If the game is lost/won at a position, it is terminated, ie, the corresponding node has no children.

- Nodes at even levels (the root is at level 0) are the positions when the first player (call her A) is to play and nodes at the odd levels are the positions when the second player (call her B) is to play.

- Now, we say a node is winning iff there exists a strategy for the player of that node such that no matter what the other player does, she will win. A node is losing iff there if no matter what the player of the node does, the other player can play in such a way that she loses. If neither are true, we say the node is drawn.

- Now, if a node is winning, then there must exist a move the node player makes such that from then on the other player is losing, even if she plays perfectly. A move is a transition from a parent node to a child node (remember parent player and child player are different).

- - So, a node is winning if there exists a child node that is losing.
- - A node is drawn if there exists no losing child node but there does exist a drawn child node.
- - A node is losing if every child node is winning.

- Since we know the statuses of all the leaf nodes (all either winning, losing or drawn), we can get the statuses of every node (ie every position) from the bottom up, recursively. Hence, every node is either winning, losing or drawn, including the root node.

- This proves the statement above, and this algorithm for finding the node statuses is known as the Minimax algorithm.

- The disadvantage is that this algorithm is extremely computationally expensive. It's training time (time taken to find out status of each node) is proportional to the number of nodes, ie, the number of reachable game states. For a game like Chess, this is on the order of 10^40!

- Therefore, the minimax algorithm is not really used, except for tiny problems, and instead, real reinforcement learning algorithms are used, which may be less accurate, but are way less computationally expensive.

In [66]:
'''
For playing TicTacToe run all the cells and wait till the model is trained
You will be asked to enter number corresponding to the boards where you want to make your move, for example in 1 3x3 TicTacToe:
1 | 2 | 3
4 | 5 | 6
7 | 8 | 9
The model trains for a 3x3 TicTacToe by default, you can definitely modify the values(of N, number of iterations etc) for your convenience but training model for bigger N will take lots of time
'''

'\nFor playing TicTacToe run all the cells and wait till the model is trained\nYou will be asked to enter number corresponding to the boards where you want to make your move, for example in 1 3x3 TicTacToe:\n1 | 2 | 3\n4 | 5 | 6\n7 | 8 | 9\nThe model trains for a 3x3 TicTacToe by default, you can definitely modify the values(of N, number of iterations etc) for your convenience but training model for bigger N will take lots of time\n'

In [67]:
import numpy as np
import copy

In [68]:
class TicTacToeEngine:
    '''
    Write a Python program that can play an NxN game of Tic-Tac-Toe optimally against a human player (a Tic-Tac-Toe engine, essentially).

    The Tic-Tac-Toe engine should have already learnt the optimal strategy for playing NxN Tic-Tac-Toe via the Minimax Algorithm.

    Again, add any helper classes you require.
    '''
    def __init__(self, N, board, turn):
        self.N = N
        self.board = board
        self.turn = turn
        if self.win_check() == 2:
            self.create_childs()
    
    def win_check(self):
        list_row = np.sum(self.board, axis=1)
        list_col = np.sum(self.board, axis=0)
        list_diag = [sum([self.board[i][i] for i in range(self.N)]), sum([self.board[i][self.N-i-1] for i in range(self.N)])]
        if np.any(list_row==self.N) or np.any(list_col==self.N) or self.N in list_diag:
            return 1
        elif np.any(list_row==-self.N) or np.any(list_col==-self.N) or -self.N in list_diag:
            return -1
        if np.all(self.board != 0):
            return 0
        return 2

    def create_childs(self):
        self.list_child = []
        for i in range(self.N):
            for j in range(self.N):

                if self.board[i][j] == 0:
                    buf_board = np.copy(self.board)

                    buf_board[i][j] = -self.turn
            
                    buf_state = TicTacToeEngine(self.N, buf_board, -self.turn)
                    self.list_child.append(buf_state)
        self.list_vals = [child_states.show_vals() for child_states in self.list_child]

    def show_board(self):
        return self.board
    
    def show_childs(self):
        return self.list_child
    
    def show_list_vals(self):
        try:
            return self.list_vals
        except:
            return None
        
    def show_vals(self):
        if self.win_check() == 1:
            return [1,0]
        elif self.win_check() == -1:
            return [0,1]
        elif self.win_check() == 0:
            return [0,0]
        elif self.turn == -1:
            self.player1 = max([self.list_vals[i][0] for i in range(len(self.list_vals))])
            self.player2 = min([self.list_vals[i][1] for i in range(len(self.list_vals))])
        elif self.turn == 1:
            self.player1 = min([self.list_vals[i][0] for i in range(len(self.list_vals))])
            self.player2 = max([self.list_vals[i][1] for i in range(len(self.list_vals))])
        return [self.player1, self.player2]
    
    def move_maker(self, cur_board):
        buf_state = TicTacToeEngine(self.N, cur_board, -self.turn)
        if buf_state.win_check()==2:
            for i in range(len(self.list_child)):
                if np.all((self.list_child[i].show_board()==cur_board)==True):
                    choice_vals=self.list_child[i].show_list_vals()
                    pos=0
                    max=0
                    min=1
                    for j in range(len(choice_vals)):
                        if choice_vals[j][1]>max:
                            max=choice_vals[j][1]
                            min=choice_vals[j][0]
                            pos=j
                        elif choice_vals[j][1]==max and choice_vals[j][0]<min:
                            max=choice_vals[j][1]
                            min=choice_vals[j][0]
                            pos=j
                    return self.list_child[i].show_childs()[pos]
        else:
            return buf_state
    
    def reset(self):
        self.board = np.zeros((self.N, self.N))

In [69]:
def print_board(board):
    for i in board:
        str = "|"
        for j in i:
            if j == 1:
                str = str + "X|"
            elif j == -1:
                str = str + "O|"
            else:
                str = str + " |"
        print(str)
        print("------")

def game(N=3):
    N=3
    board = np.zeros((N,N))
    cur_state = TicTacToeEngine(N, board, -1)
    while cur_state.win_check()==2:
        print("Your move : ")
        response = int(input())
        response = response - 1
        response_board = np.copy(cur_state.show_board())
        response_board[response//N][response%N] = 1
        cur_state=cur_state.move_maker(response_board)
        cur_board = cur_state.show_board()
        print()
        print_board(response_board)
        print("\nComputer's move : ")
        print_board(cur_board)
        print("")
    if cur_state.win_check()==0:
        print("It's a Draw")
    elif cur_state.win_check()==1:
        print("X won!")
    elif cur_state.win_check()==-1:
        print("O won!")

In [70]:
# Starts the game ;-;
game()

Your move : 

| | | |
------
| |X| |
------
| | | |
------

Computer's move : 
|O| | |
------
| |X| |
------
| | | |
------

Your move : 

|O| | |
------
| |X| |
------
| | |X|
------

Computer's move : 
|O| |O|
------
| |X| |
------
| | |X|
------

Your move : 

|O|X|O|
------
| |X| |
------
| | |X|
------

Computer's move : 
|O|X|O|
------
| |X| |
------
| |O|X|
------

Your move : 

|O|X|O|
------
| |X|X|
------
| |O|X|
------

Computer's move : 
|O|X|O|
------
|O|X|X|
------
| |O|X|
------

Your move : 

|O|X|O|
------
|O|X|X|
------
|X|O|X|
------

Computer's move : 
|O|X|O|
------
|O|X|X|
------
|X|O|X|
------

It's a Draw
